Let $X = (X_1, \dots, X_n)$ be a sample of real-valued random variables. Suppose the distribution of the statistic
\begin{equation}
    T(X) = \max \{X_1, \dots, X_n\}
\end{equation}
is of interest. For this very simple statistic, it is possible to calculate $\sigma(T; \hat{F})$ exactly.

Let the ordered values of our observed sample be $X_{(1)} \leq \dots \le X_{(n)}$. A bootstrap sample $Y = (Y_1, \dots, Y_n)$ is created by drawing $n$ values from $\{X_1, \dots, X_n\}$ with replacement. The statistic $T(Y) = \max\{Y_1, \dots, Y_n\}$ can only take values from the set of original observations.

The cumulative distribution function of $T(\mathbf{Y})$ under $\hat{F}$ is given by
\begin{equation}
    \Pr(T(Y) \leq X_{(k)}) = \Pr(Y_i \le X_{(k)})^n,
\end{equation}
where we have used independence of $Y_i$.
Since each $Y_i$ is chosen independently from the $n$ available values, and exactly $k$ of those values are less than or equal to $X_{(k)}$, we have
\begin{equation}
    \Pr(T(Y) \leq X_{(k)}) = \left( \frac{k}{n} \right)^n.
\end{equation}
The probability that the maximum of a bootstrap sample is exactly equal to the $k$-th order statistic $X_{(k)}$ is
\begin{equation}
    p_k = Pr(T(Y) = X_{(k)}) = \Pr(T(Y) \leq X_{(k)}) - \Pr(T(Y) \leq X_{(k-1)}) = \left( \frac{k}{n} \right)^n - \left( \frac{k-1}{n} \right)^n
\end{equation}
for $k = 1, \dots, n$.

The standard error $\sigma(T; \hat{F})$ is the square root of the variance of this discrete distribution. We have
\begin{align}
    E_{\hat{F}}[T(Y)] &= \sum_{k=1}^n X_{(k)} p_k = \sum_{k=1}^n X_{(k)} \left[ \left( \frac{k}{n} \right)^n - \left( \frac{k-1}{n} \right)^n \right], \\
    E_{\hat{F}}[T(Y)^2] &= \sum_{k=1}^n X_{(k)}^2 p_k = \sum_{k=1}^n X_{(k)}^2 \left[ \left( \frac{k}{n} \right)^n - \left( \frac{k-1}{n} \right)^n \right],
\end{align}
and the exact bootstrap standard error is
\begin{equation}
    \sigma(T; \hat{F}) = \sqrt{E_{\hat{F}}[T(Y)^2] - \left( E_{\hat{F}}[T(Y)] \right)^2}.
\end{equation}

---

Suppose the sample $X$ comes from the uniform distribution on $[0, \theta]$ for some $\theta > 0$. Then $T(X)$ is the maximum likelihood estimator for $\theta$. The CDF of a single $X_i$ is $F(x) = x/\theta$ for $x \in [0, \theta]$. The CDF of the maximum $T$ is
\begin{equation}
    F_T(t) = \Pr(T \leq t) = \Pr(X_1 \leq t, \dots, X_n \leq t) = F(t)^n = \left(\frac{t}{\theta}\right)^n
\end{equation}
for $0 \leq t \leq \theta$ and the PDF of $T$ is the derivative
\begin{equation}
    f_T(t) = \frac{d}{dt} F_T(t) = \frac{n t^{n-1}}{\theta^n}.
\end{equation}
Hence the moments can be computed as
\begin{align}
    E[T] &= \int_0^\theta t \frac{n t^{n-1}}{\theta^n} \,dt = \frac{n}{\theta^n} \left[ \frac{t^{n+1}}{n+1} \right]_0^\theta = \frac{n}{n+1} \theta, \\
    E[T^2] &= \int_0^\theta t^2 \frac{n t^{n-1}}{\theta^n} \,dt = \frac{n}{\theta^n} \left[ \frac{t^{n+2}}{n+2} \right]_0^\theta = \frac{n}{n+2} \theta^2.
\end{align}
which gives the formula for the variance and standard error,
\begin{align}
    Var(T)
    &= E[T^2] - (E[T])^2 = \frac{n}{n+2}\theta^2 - \left(\frac{n}{n+1}\theta\right)^2 \\
    &= \theta^2 \left[ \frac{n(n+1)^2 - n^2(n+2)}{(n+2)(n+1)^2} \right] \\
    &= \frac{n\theta^2}{(n+1)^2(n+2)}.
\end{align}
Thus, the standard error is
\begin{equation}
    \sigma(T; F) = \frac{\theta \sqrt{n}}{(n+1)\sqrt{n+2}}
\end{equation}


In [1]:
import numpy as np

def exact_bootstrap_se_max(data):
    x_sorted = np.sort(data)
    n = len(x_sorted)
    k = np.arange(1, n + 1)
    p = (k/n)**n - ((k-1)/n)**n
    e_val = np.sum(x_sorted * p)
    e_val_sq = np.sum((x_sorted**2) * p)
    return np.sqrt(e_val_sq - e_val**2)

theta = 5
ns = [100, 500, 1000, 5000]

for n in ns:
    X = np.random.uniform(0, theta, n)

    se_true = (theta * np.sqrt(n)) / ((n+1) * np.sqrt(n+2))
    se_boot = exact_bootstrap_se_max(X)

    print(f"n={n:5} | True SE: {se_true:.6f} | Bootstrap SE: {se_boot:.6f} | Ratio: {se_boot/se_true:.3f}")

n=  100 | True SE: 0.049017 | Bootstrap SE: 0.036482 | Ratio: 0.744
n=  500 | True SE: 0.009960 | Bootstrap SE: 0.006105 | Ratio: 0.613
n= 1000 | True SE: 0.004990 | Bootstrap SE: 0.010846 | Ratio: 2.174
n= 5000 | True SE: 0.001000 | Bootstrap SE: 0.000824 | Ratio: 0.824


The bootstrap method performs poorly for the sample maximum. The ratio of the bootstrap SE to the true SE remains significantly different from $1$ even as $n$ increases. It systematically underestimates the true variability of the maximum. The bootstrap fails here due to lack of smoothness at the boundary of the distribution.

1.  The empirical distribution $\hat{F}$ has a jump at the observed maximum $X_{(n)}$. In any bootstrap sample, there is a probability of approximately $1 - e^{-1} \approx 0.632$ that the bootstrap maximum will be exactly $X_{(n)}$.

2.  The maximum statistic is sensitive to the boundary of the distribution as the bootstrap relies on re-sampling the edge but it can never see beyond the current maximum. Since the true population $F$ continues up to $\theta$, the true sampling distribution of $T$ is continuous and spread out toward $\theta$.

3.  Mathematically, the bootstrap distribution of $n(\theta - T(Y))$ does not converge to the true limiting distribution of $n(\theta - T(X))$, i.e., it is inconsistent. Because the bootstrap stays at the observed maximum so often, it fails to capture the full range of potential variation that a real new sample would have.